In [1]:
###### In this notebook we're going to compute the overlap metrics between T5 and the baselines ######

In [50]:
import pandas as pd
import re

counter_pred = 0

mispred_list = []

sanity_check_list = []

idx = 0

BEAM_SIZE = 1


### LOAD REFERENCES AND T5 PREDICTIONS ####

#Let's import the ground truth from the test dataset
df = pd.read_csv('/Users/antonio/Desktop/T5_project/datasets/fine-tuning/CS/test.tsv',header=None,sep='\t')

references=[]

for item in df[1]:
    references.append(item.lower())


perfect_pred_t5 = []
pred_refined_t5=[]
pred_refined_t5_to_set = []

with open('/Users/antonio/Desktop/ICSE_PRED/T5/CS_predictions@1.txt') as fread:
    for item in fread.readlines():
        item = item.strip()
        if len(item)>=2:
          if item[0]=='"':
              item = item[1:]
          if item[-1]=='"':
              item = item[:-1]
                
        pred_refined_t5.append(item)
        
    
len_prediction=(len(pred_refined_t5))


def cleanCommentCSTask(comment):
    comment = comment[0]
    target_comment = re.findall("<s>([\s\S]*?)</s>", comment)
    if len(target_comment)>0: return target_comment[0].strip().lower()
    else: return ''

for i in range(0, len_prediction, BEAM_SIZE):

    items_to_analyze = pred_refined_t5[i:i+BEAM_SIZE]
    target_item = ''.join(references[idx].split(' '))
    

    for pred in items_to_analyze:
        pred_ref = ''.join(pred.split(' '))
        if pred_ref == target_item:
            perfect_pred_t5.append(items_to_analyze)
            counter_pred+=1
            sanity_check_list.append(pred_ref)
            pred_refined_t5_to_set.append(pred_ref)
            break
       
        
    idx += 1

percentage = (counter_pred/len(references))*100 
print('[T5] % of perfect predictions: {} {}/{}'.format(percentage, counter_pred, len(references)))


with open('CS_perfect@1.txt','a+') as fwrite:
    for pred in perfect_pred_t5:
        fwrite.write(pred[0]+'\n')


############# BASELINE ##########

counter_pred = 0

mispred_list = []

sanity_check_list = []

idx = 0

pred_refined_baseline=[]
pred_refined_baseline_to_set = []

with open('/Users/antonio/Desktop/ICSE_PRED/Baselines/CS/predictions.txt') as fread:
    for item in fread.readlines():
        item = item.strip().lower()
        pred_refined_baseline.append(item)
        

### UNCOMMENT ONLY FOR BFmedium since the Test dataset is not in the same order(row-wise) of the original one ######
# references=[]

# with open('/Users/antonio/Downloads/fixed.txt') as fread:
#     for item in fread.readlines():
#         references.append(item.strip().lower())
        
    
for i in range(0, len_prediction, BEAM_SIZE):
    
    #### Instrumentation for CS task ####
    #### Keep COMMENTED ####
    
#     if len(pred_refined_baseline[i:i+BEAM_SIZE])==0:
#         continue
#     else:  
#         items_to_analyze = [cleanCommentCSTask(pred_refined_baseline[i:i+BEAM_SIZE])]
    
    items_to_analyze = pred_refined_t5[i:i+BEAM_SIZE]
    target_item = ''.join(references[idx].split(' '))
    

    for pred in items_to_analyze:
        pred_ref = ''.join(pred.split(' '))
        if pred_ref == target_item:
            counter_pred+=1
            sanity_check_list.append(pred_ref)
            pred_refined_baseline_to_set.append(pred_ref)
            break
       
        
    idx += 1

percentage = (counter_pred/len(references))*100 
print('[Baseline] % of perfect predictions: {} {}/{}'.format(percentage, counter_pred, len(references)))

[T5] % of perfect predictions: 11.441237294847538 10401/90908
[Baseline] % of perfect predictions: 3.3528402340828087 3048/90908


In [51]:
###### OVERLAP #####

set_t5 = set(pred_refined_t5_to_set)
set_baseline = set(pred_refined_baseline_to_set)

#### SHARED #####
intersection = set_t5.intersection(set_baseline)
union = set_t5.union(set_baseline)
shared = (len(intersection)/len(union) ) * 100

print('Shared: ',shared)
# print(len(intersection))

#### ONLY T5 #####
difference = set_t5.difference(set_baseline)
union = set_t5.union(set_baseline)
t5_only = (len(difference)/len(union) ) * 100

print('Only T5: ',t5_only)
# print(len(intersection))

#### ONLY BASELINE #####
difference = set_baseline.difference(set_t5)
union = set_t5.union(set_baseline)
baseline_only = (len(difference)/len(union) ) * 100

print('Only BASELINE: ',baseline_only)

assert(t5_only + shared + baseline_only > 99.9)

Shared:  4.966329966329966
Only T5:  93.46801346801347
Only BASELINE:  1.5656565656565657
